<a href="https://colab.research.google.com/github/Pinokcio/ML_Study/blob/main/practice/Cosine_similarity_recsys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Cosine_Similarity를 이용한 추천 시스템

#데이터 입력

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive
%cd MyDrive
%ls

f = open("1.txt", 'r')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
num_sim_user_top_N = int(f.readline())
num_item_rec_top_M = int(f.readline())
num_users = int(f.readline())
num_items = int(f.readline())
num_rows = int(f.readline())
board = [[None for _ in range(num_items)] for _ in range(num_users)] #board[i][j] : i유저가 j아이템에 부여한 점수

for i in range(num_rows):
  user, item, score = f.readline().split()
  board[int(user)-1][int(item)-1] = float(score)
num_reco_users = int(f.readline())
list = [0 for _ in range(num_reco_users)]
for i in range(num_reco_users):
  list[i] = int(f.readline())

#유사도 계산

In [ ]:
import pandas as pd
import math
df = pd.DataFrame(board)
df = df.fillna(0)
similarity_rate = [[0 for _ in range(num_users)] for _ in range(num_users)]
for a in range(num_users):
  for b in range(a+1, num_users):
    tmp_a, tmp_b, tmp = 0, 0, 0
    for i in range(num_items):
      tmp_a += pow(df.iloc[a, i], 2)
      tmp_b += pow(df.iloc[b, i], 2)
      tmp += df.iloc[a, i] * df.iloc[b, i]
    tmp_a = math.sqrt(tmp_a)
    tmp_b = math.sqrt(tmp_b)
    if tmp_a == 0 or tmp_b == 0:
      similarity_rate[a][b] = 0.0
      similarity_rate[b][a] = 0.0 
    else:
      similarity_rate[a][b] = tmp/(tmp_a*tmp_b)
      similarity_rate[b][a] = tmp/(tmp_a*tmp_b)

similarity_rate_df = pd.DataFrame(data=similarity_rate,index=df.index, columns=df.index)
print(similarity_rate_df)
print(df)

          0         1         2         3         4         5         6    \
0    0.000000  0.439047  0.579556  0.421390  0.238610  0.426254  0.579443   
1    0.439047  0.000000  0.414212  0.261249  0.047659  0.492335  0.330324   
2    0.579556  0.414212  0.000000  0.620727  0.507489  0.394532  0.602945   
3    0.421390  0.261249  0.620727  0.000000  0.293311  0.146301  0.369125   
4    0.238610  0.047659  0.507489  0.293311  0.000000  0.198491  0.283078   
..        ...       ...       ...       ...       ...       ...       ...   
195  0.247145  0.123755  0.401415  0.470193  0.323552  0.257404  0.255169   
196  0.480610  0.205951  0.589717  0.596702  0.470709  0.318495  0.458834   
197  0.310803  0.126712  0.220647  0.248802  0.091730  0.243911  0.162271   
198  0.423515  0.035988  0.275278  0.130435  0.365311  0.156284  0.019134   
199  0.601210  0.241576  0.285710  0.303560  0.040003  0.245299  0.300682   

          7         8         9    ...       190       191       192  \
0  

In [ ]:
"""import pandas as pd
df = pd.DataFrame(board)
df = df.fillna(0)
similarity_rate = cosine_similarity(df, df)
similarity_rate_df = pd.DataFrame(data=similarity_rate,index=df.index,columns=df.index)
print(df)"""

'import pandas as pd\ndf = pd.DataFrame(board)\ndf = df.fillna(0)\nsimilarity_rate = cosine_similarity(df, df)\nsimilarity_rate_df = pd.DataFrame(data=similarity_rate,index=df.index,columns=df.index)\nprint(df)'

#NDCG 계산 및 결과 출력

In [ ]:
import numpy as np
np.fill_diagonal(similarity_rate_df.values, 0)

In [ ]:
nearest_n = [[[similarity_rate_df.iloc[i,j], j] for j in range(num_users)] for i in range(num_users)]
for n in range(len(nearest_n)):
  nearest_n[n].sort(reverse=True)

nearest_k = [[nearest_n[i][j][1] for j in range(num_sim_user_top_N)] for i in range(num_users)]

In [ ]:
item_top_m = []
r = [0 for i in range(num_users)]
for i in range(num_users):
  tmp = 0
  cnt = 0
  for j in range(num_items):
    if df.iloc[i, j] != 0:
      tmp += df.iloc[i, j]
      cnt += 1
  if cnt != 0:
    r[i] = tmp/cnt  

k = [0 for i in range(num_users)]
for i in range(num_users):
  tmp = 0
  for j in nearest_k[i]:
    tmp += abs(similarity_rate_df.iloc[i,j])
  if tmp != 0:
    k[i] = 1/tmp

recom_item = [[] for i in range(num_users)]
for u in range(num_users):
  for i in range(num_items):
    if df.iloc[u,i] == 0:
      tmp = 0
      for n in nearest_k[u]:
        tmp += similarity_rate_df.iloc[u, n]*(df.iloc[n,i]-r[n])
      tmp *= k[n]
      tmp += r[u]
      recom_item[u].append([tmp ,i+1])
for i in range(len(recom_item)):
  recom_item[i] = sorted(recom_item[i], reverse = True)[:num_item_rec_top_M]
  for j in range(len(recom_item[i])):
    recom_item[i][j] = recom_item[i][j][1]
#print(recom_item)

In [ ]:
import math
def ndcg(gt, rec):
    idcg = sum([1.0 / math.log(i + 2, 2) for i in range(len(gt))])
    dcg = 0.0
    for i, r in enumerate(rec):
        if r not in gt:
            continue
        gt_index = gt.index(r)
        if i != gt_index:
            rel = 0.7
        else:
            rel = 1.0
        dcg += rel / math.log(i + 2, 2)
    ndcg = dcg / idcg

    return ndcg
f_out = open("2out.txt", 'r')
output = [[0 for _ in range(num_item_rec_top_M)] for _ in range(num_reco_users)]
for i in range(num_reco_users):
  output[i] = [int(i) for i in f_out.readline().split()]
ans = 0
cnt = 0
for e, l in enumerate(list):
  print(recom_item[l-1], output[e])
  print(ndcg(recom_item[l-1], output[e]))
  if ndcg(recom_item[l-1], output[e]) >= 0.9:
    ans += 1
  cnt += 1
print('score: ', ans/cnt)
